# Logic Test Notebook
`run.py`의 실행 로직을 기반으로 특정 프로젝트의 메인 로직을 테스트합니다.

In [ ]:
import os
import sys
import importlib
from pkg.utils.blackboard import initialize_global_blackboard, GlobalBlackboard
import time

# 1. 프로젝트 루트 경로 설정 (노트북 위치 기준)
project_root = os.path.abspath('')
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project Root: {project_root}")

# 2. 실행할 프로젝트 이름 설정
PROJECT_NAME = "shimadzu_logic"

# 3. Blackboard 초기화 (run.py와 동일한 경로 규칙 사용) 
blackboard_path = os.path.join(project_root, f"projects/{PROJECT_NAME}/configs/blackboard.json")

if os.path.exists(blackboard_path):
    initialize_global_blackboard(blackboard_path)
    print(f"✅ Blackboard initialized from: {blackboard_path}")
else:
    print(f"❌ Blackboard config not found at: {blackboard_path}")

# 4. 프로젝트 메인 모듈 로드 및 실행
module_name = f"projects.{PROJECT_NAME}.main"

try:
    # 소스 코드 수정 시 즉시 반영하기 위해 reload 처리
    if module_name in sys.modules:
        module = importlib.reload(sys.modules[module_name])
    else:
        module = importlib.import_module(module_name)

    if hasattr(module, "main"):
        print(f"🚀 Starting {module_name}.main()...\n")
        module.main()
        bb = GlobalBlackboard()
    else:
        print(f"⚠️ Error: Module '{module_name}' does not define main()")
except Exception as e:
    print(f"❌ Error running project: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
robot_s = bb.get("robot/fsm/strategy")
device_s = bb.get("device/fsm/strategy")
logic_s = bb.get("logic/fsm/strategy")
auto_cmd = bb.get("ui/cmd/auto/tensile")

print(f"robot {robot_s}\ndevice {device_s}\nlogic {logic_s}\nauto cmd {auto_cmd}")

In [ ]:
bb.get("process_status/thickness_measurement")

In [ ]:
bb.set("manual/device/tester",12)

In [ ]:
# 인디게이터 두께 측정
bb.set("manual/device/tester",9)
time.sleep(2)
bb.get("device/gauge/thickness")

In [ ]:
# 두께 측정기 받침 올리기
bb.set("manual/device/tester",12)

In [ ]:
# 두께 측정기 받침 내리기
bb.set("manual/device/tester",13)

In [ ]:
# 시편 정렬기 풀기
bb.set("manual/device/tester",7)

In [ ]:
# 시편 정렬기 시편 정렬
bb.set("manual/device/tester",6)

In [ ]:
bb.get("device/remote/input/SOL_SENSOR")

In [ ]:
bb.get("process_status")

In [ ]:
# setting화면 보낸 데이터
bb.get("system/setting/context")

In [ ]:
data = {
    "batch_id" : "BT-20251226",
    "status" : "시험중"
}
bb.set("process_status/batch_info",data)

In [ ]:
bb.get("process_status/batch_info")

In [ ]:
runtime = {
    "starttime" : "16:39:00",
    "elapsedtime" : "05:13:12"
}
bb.set("process_status/runtime",runtime)

In [ ]:
bb.get("process_status/runtime")

In [ ]:
robot_s = bb.get("robot/fsm/strategy")
device_s = bb.get("device/fsm/strategy")
logic_s = bb.get("logic/fsm/strategy")

print(f"robot {robot_s}\ndevice {device_s}\nlogic {logic_s}")